In [ ]:
using LoopVectorization
using Plots

function myConv(x, y)
    z = zeros(typeof(x[1]), length(x) + length(y) - 1)
    @turbo for i = 1:length(x)
        for j = 1:length(y)
            z[i+j-1] += x[i] * y[j]
        end
    end
    return z
end

In [ ]:
t, h = [0, 2, 0], [1, 0, 1];
t, h = BigInt.(t), BigInt.(h);
for i = 3:100
    t, h = myConv(t + h, [0, 1, 0]), myConv(t, [0, 0, 1]) + myConv(h, [1, 0, 0])
end
a = t + h;
total = BigInt(2)^100
display(plot(-99:1:99, a / total, xlim=(-50, 50)))
sum(a[1:end÷2] / total), a[end÷2+1] / total, sum(a[end÷2+2:end] / total)

In [ ]:
function myDTFT(x, n=256)
    coords = range(0, 2π, length=n + 1)[1:end-1]
    exp.(-1im * coords .* (-length(x)÷2:length(x)÷2)') * x / 2
end

function myIDTFT(z)
    coords = range(0, 2π, length=length(z) + 1)[1:end-1]
    i = -length(z)÷2:length(z)÷2
    exp.(1im * i .* coords') * z / length(z) * 2
end

t, h = [0, 2, 0], [1, 0, 1];
dt, dh = myDTFT(t) / 2, myDTFT(h) / 2
dk1, dk2 = myDTFT([0, 0, 1]), myDTFT([1, 0, 0])
for i = 3:7
    dt, dh = (dt + dh) / 2, dt .* dk1 + dh .* dk2
end
r = real(myIDTFT((dt .+ dh) / 2))
display(plot(r))
r[end÷2+1]

In [ ]:
using Einsum
coords = range(0, 2π, length=256 + 1)[1:end-1]
x = cat(ones(size(coords)), cos.(coords); dims=2) / 2
k1, k2 = exp.(-coords * im) / 2, exp.(coords * im) / 2
mat = reshape(cat(fill(1 / 2, size(k1)), fill(1 / 2, size(k2)), k1, k2; dims=2), (length(k1), 2, 2))
for i = 3:100
    @einsum u[i, j] := mat[i, r, j] * x[i, r]
    x = u
end
r = real(myIDTFT(x * ones(2) / 2))
display(plot(r))
r[end÷2+1]

In [ ]:
# using Symbolics, Polynomials
# using Groebner

# sqrtrule2 = @rule sqrt(~x)^2 => ~x
# sqrtrule4 = @rule sqrt(~x)^4 => ~x^2
# chain = Rewriters.Chain([sqrtrule2, sqrtrule4])
# rewriter = Rewriters.Prewalk(chain)

# # @variables w, λ
# coords = range(0, 2π, length=256 + 1)[1:end-1]
# x = cat(ones(size(w)), cos.(w); dims=1) / 2
# k1, k2 = exp.(-w * im) / 2, exp.(w * im) / 2
# mat = reshape(cat(fill(1 / 2 + 0im, size(k1)), fill(1 / 2 + 0im, size(k2)), k1, k2; dims=2), (2, 2)) * 2
# # poly = (mat[1, 1] - λ) * (mat[2, 2] - λ) - mat[1, 2] * mat[2, 1]
# # expand(poly).coeffs
# HH, i = mat, 1
# Hii = HH[i, i]
# Hi1i1 = HH[i+1, i+1]
# d = Hii * Hi1i1 - HH[i, i+1] * HH[i+1, i]
# t = Hii + Hi1i1
# x = 0.5 * t
# # y = complex(x * x - d)^(0.5)
# # x^2 - d = −0.25sin^2(w)+0.25(1.0+cos(w))^2+(sin(−w)−sin(w)+0.5sin(w)(1.0+cos(w)))i


In [16]:
using SymPy

@syms w, λ
coords = range(0, 2π, length=256 + 1)[1:end-1]
x = cat(ones(size(w)), cos.(w); dims=1) / 2
k1, k2 = exp.(-w * im) / 2, exp.(w * im) / 2
mat = reshape(cat(fill(1 / 2 + 0im, size(k1)), fill(1 / 2 + 0im, size(k2)), k1, k2; dims=2), (2, 2))
P, D = mat.diagonalize()
values = sum(P * (D^98) * P^-1 * x)
dtft_under_integral = values * exp(-w * im) / 2π
solution = integrate(dtft_under_integral, (w, 0, 2π))
solution